In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0


# Tables

---

## kinesis source
- clickstream

## ingested tables
- clickstream_raw

## bronze tables
- clickstream_bronze

In [0]:
from config.settings import *

In [0]:
%run "../../config/setup"

In [0]:

streaming_df = (spark.readStream \
    .format("kinesis") \
    .option("streamName", KINESIS_STREAM) \
    .option("region", AWS_REGION) \
    .option("initialPosition", "latest") \
    .load())
    

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BinaryType

clickstream_schema = StructType([
    StructField("partitionKey", StringType(), True),
    StructField("data", BinaryType(), True),
    StructField("stream", StringType(), True),
    StructField("shardId", StringType(), True),
    StructField("sequenceNumber", StringType(), True),
    StructField("approximateArrivalTimestamp", TimestampType(), True)
])

parsed = (streaming_df \
    .withColumn("json", F.from_json(F.col("data").cast("string"), clickstream_schema)) \
    .selectExpr("json.*"))

display(parsed)
# query = (parsed
#     .writeStream \
#     .format("delta") \
#     .option("checkpointLocation", f"{S3_BUCKET_URI}/checkpoints") \
#     .outputMode("append") \
#     .trigger(availableNow=True) \
#     .toTable("clickstream_raw"))

# query.awaitTermination()